In [1]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb

In [2]:
path='./'
train = pd.read_csv(path+'input/train_set.csv')
test = pd.read_csv(path+'input/test_set.csv')
print(train.info())
print(test.info())

#如何从64下降到32

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
ID           25317 non-null int64
age          25317 non-null int64
job          25317 non-null object
marital      25317 non-null object
education    25317 non-null object
default      25317 non-null object
balance      25317 non-null int64
housing      25317 non-null object
loan         25317 non-null object
contact      25317 non-null object
day          25317 non-null int64
month        25317 non-null object
duration     25317 non-null int64
campaign     25317 non-null int64
pdays        25317 non-null int64
previous     25317 non-null int64
poutcome     25317 non-null object
y            25317 non-null int64
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 17 columns):
ID           10852 non-null int64
age          10852 non-null int64
job          10852 non-null object
mar

In [3]:
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [4]:
test.describe() #描述统计

,ID,age,balance,day,duration,campaign,pdays,previous
count,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000
mean,30743.500000,41.040638,1393.157298,15.680151,257.206137,2.770365,39.954755,0.546443
std,3132.846895,10.652369,3318.497101,8.302317,250.480906,3.063481,99.524056,1.805938
min,25318.000000,18.000000,-2604.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,28030.750000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,30743.500000,39.000000,450.000000,16.000000,181.000000,2.000000,-1.000000,0.000000
75%,33456.250000,49.000000,1440.250000,21.000000,322.000000,3.000000,-1.000000,0.000000
max,36169.000000,94.000000,81204.000000,31.000000,3102.000000,58.000000,871.000000,51.000000


In [5]:
train.corr() #相关分析

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [6]:
train.job.value_counts() #查看job的分布，发现这个数据集，这个职业比较稳定

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [7]:
test['y']=-1 #test和train的列是一样的
print(len(test.columns))

18


In [8]:
data = train.append(test).reset_index(drop=True) #合并数据重新排序
# data = pd.concat([train,test]).rese_index(drop=True)
print(data.head())

   ID  age         job   marital  education default  balance housing loan  \
0   1   43  management   married   tertiary      no      291     yes   no   
1   2   42  technician  divorced    primary      no     5076     yes   no   
2   3   47      admin.   married  secondary      no      104     yes  yes   
3   4   28  management    single  secondary      no     -994     yes  yes   
4   5   42  technician  divorced  secondary      no     2974     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0   unknown    9   may       150         2     -1         0  unknown  0  
1  cellular    7   apr        99         1    251         2    other  0  
2  cellular   14   jul        77         2     -1         0  unknown  0  
3  cellular   18   jul       174         2     -1         0  unknown  0  
4   unknown   21   may       187         5     -1         0  unknown  0  


In [10]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
#os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [11]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder() #了解LableEncoder把类别转换成数值
    data['count_' + i] = data.groupby([i])[i].transform('count') #计数的特征工程
    data[i] = lbl.fit_transform(data[i].astype(str)) #把类别转换成数值

In [12]:
data.head() #查看

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,...,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,...,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,...,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,...,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,...,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


1ID Int客户唯一标识

2age Int客户年龄

3job String客户的职业

4marital String婚姻状况

5education String受教育水平

6default String是否有违约记录

7balance Int每年账户的平均余额

8housing String是否有住房贷款

9loan String是否有个人贷款

10contact String与客户联系的沟通方式

11day Int最后一次联系的时间（几号）

12month String最后一次联系的时间（月份）

13duration Int最后一次联系的交流时长

14campaign Int在本次活动中，与该客户交流过的次数

15pdays Int距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过）16previousInt在本次活动之前，与该客户交流过的次数

17poutcome String上一次活动的结果

18y Int预测客户是否会订购定期存款业务

In [13]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,...,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,...,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,...,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,...,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,...,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


In [14]:
feats = [i for i in data.columns if i not in ['ID','y']] #Y是预测的没有当成特征
feats

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'count_job',
 'count_marital',
 'count_education',
 'count_default',
 'count_housing',
 'count_loan',
 'count_contact',
 'count_month',
 'count_poutcome']

In [15]:
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )
train_x =data[data['y']!=-1][feats]
train_y =data[data['y']!=-1]['y'] #
testx= data[data['y']==-1][feats]
# train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

#概率预测

In [16]:
print(len(test_pre))

10852


In [17]:
pre=data[data['y']==-1][['ID']] #找到y=-1的ID
pre['pred']=test_pre
print(len(pre))

10852


In [18]:
pre.head()

,ID,pred
25317,25318,0.053728
25318,25319,0.003968
25319,25320,0.001854
25320,25321,0.785342
25321,25322,0.013772


In [23]:
pre.to_csv('baseline2.csv',index=False )

In [25]:
#10折交叉验证，跑10次
from sklearn.model_selection import KFold
n_splits=10
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']
res=data[data['y']==-1][['ID']]

test_x= data[data['y']==-1][feats]
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):  #进行kfold迭代
    model.random_state = model.random_state + 1  #每次种子+1
    train_x1 = train_x.loc[train_idx]             #训练里面的x1
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/5 #累积完除以10
res.to_csv('./submission10_fold.csv', index=False)

[1]	training's auc: 0.891388	valid_1's auc: 0.880231
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.909563	valid_1's auc: 0.897005
[3]	training's auc: 0.921438	valid_1's auc: 0.906902
[4]	training's auc: 0.923407	valid_1's auc: 0.908966
[5]	training's auc: 0.924011	valid_1's auc: 0.909937
[6]	training's auc: 0.924636	valid_1's auc: 0.912778
[7]	training's auc: 0.926612	valid_1's auc: 0.913687
[8]	training's auc: 0.927152	valid_1's auc: 0.914268
[9]	training's auc: 0.93007	valid_1's auc: 0.916217
[10]	training's auc: 0.929647	valid_1's auc: 0.916939
[11]	training's auc: 0.930108	valid_1's auc: 0.917403
[12]	training's auc: 0.930739	valid_1's auc: 0.917529
[13]	training's auc: 0.931683	valid_1's auc: 0.917664
[14]	training's auc: 0.932083	valid_1's auc: 0.918233
[15]	training's auc: 0.932388	valid_1's auc: 0.918427
[16]	training's auc: 0.932791	valid_1's auc: 0.919162
[17]	training's auc: 0.932872	valid_1's auc: 0.918982
[18]	training's auc: 0.933514	

[151]	training's auc: 0.950601	valid_1's auc: 0.931162
[152]	training's auc: 0.950659	valid_1's auc: 0.931199
[153]	training's auc: 0.950764	valid_1's auc: 0.931145
[154]	training's auc: 0.950902	valid_1's auc: 0.931345
[155]	training's auc: 0.951017	valid_1's auc: 0.931368
[156]	training's auc: 0.951095	valid_1's auc: 0.931424
[157]	training's auc: 0.951144	valid_1's auc: 0.931433
[158]	training's auc: 0.951253	valid_1's auc: 0.931448
[159]	training's auc: 0.95137	valid_1's auc: 0.931533
[160]	training's auc: 0.951478	valid_1's auc: 0.931498
[161]	training's auc: 0.951527	valid_1's auc: 0.931536
[162]	training's auc: 0.951584	valid_1's auc: 0.931505
[163]	training's auc: 0.951693	valid_1's auc: 0.931549
[164]	training's auc: 0.951777	valid_1's auc: 0.931532
[165]	training's auc: 0.951851	valid_1's auc: 0.931638
[166]	training's auc: 0.951897	valid_1's auc: 0.931664
[167]	training's auc: 0.951954	valid_1's auc: 0.931707
[168]	training's auc: 0.952028	valid_1's auc: 0.931636
[169]	train

[300]	training's auc: 0.961646	valid_1's auc: 0.933169
[301]	training's auc: 0.961716	valid_1's auc: 0.933142
[302]	training's auc: 0.961804	valid_1's auc: 0.933169
[303]	training's auc: 0.961883	valid_1's auc: 0.933173
[304]	training's auc: 0.961974	valid_1's auc: 0.933219
[305]	training's auc: 0.962058	valid_1's auc: 0.933241
[306]	training's auc: 0.962106	valid_1's auc: 0.933253
[307]	training's auc: 0.962163	valid_1's auc: 0.933247
[308]	training's auc: 0.962244	valid_1's auc: 0.933276
[309]	training's auc: 0.9623	valid_1's auc: 0.933263
[310]	training's auc: 0.962367	valid_1's auc: 0.933198
[311]	training's auc: 0.962432	valid_1's auc: 0.933232
[312]	training's auc: 0.962487	valid_1's auc: 0.93324
[313]	training's auc: 0.96257	valid_1's auc: 0.933269
[314]	training's auc: 0.962641	valid_1's auc: 0.933279
[315]	training's auc: 0.962691	valid_1's auc: 0.933279
[316]	training's auc: 0.962744	valid_1's auc: 0.933235
[317]	training's auc: 0.962801	valid_1's auc: 0.933225
[318]	training

[449]	training's auc: 0.969967	valid_1's auc: 0.933597
[450]	training's auc: 0.97	valid_1's auc: 0.933628
[451]	training's auc: 0.970046	valid_1's auc: 0.933664
[452]	training's auc: 0.970083	valid_1's auc: 0.933667
[453]	training's auc: 0.970153	valid_1's auc: 0.933727
[454]	training's auc: 0.970212	valid_1's auc: 0.933767
[455]	training's auc: 0.970271	valid_1's auc: 0.933731
[456]	training's auc: 0.970319	valid_1's auc: 0.93378
[457]	training's auc: 0.970362	valid_1's auc: 0.933746
[458]	training's auc: 0.970414	valid_1's auc: 0.933736
[459]	training's auc: 0.970459	valid_1's auc: 0.933733
[460]	training's auc: 0.97051	valid_1's auc: 0.933717
[461]	training's auc: 0.970544	valid_1's auc: 0.933721
[462]	training's auc: 0.970597	valid_1's auc: 0.933708
[463]	training's auc: 0.970637	valid_1's auc: 0.933712
[464]	training's auc: 0.970686	valid_1's auc: 0.93369
[465]	training's auc: 0.970734	valid_1's auc: 0.933674
[466]	training's auc: 0.970764	valid_1's auc: 0.93365
[467]	training's a

[598]	training's auc: 0.976443	valid_1's auc: 0.934039
[599]	training's auc: 0.976475	valid_1's auc: 0.93403
[600]	training's auc: 0.97651	valid_1's auc: 0.934035
[601]	training's auc: 0.976544	valid_1's auc: 0.93402
[602]	training's auc: 0.976577	valid_1's auc: 0.933985
[603]	training's auc: 0.976597	valid_1's auc: 0.933961
[604]	training's auc: 0.976637	valid_1's auc: 0.933986
[605]	training's auc: 0.976676	valid_1's auc: 0.934015
[606]	training's auc: 0.976744	valid_1's auc: 0.934002
[607]	training's auc: 0.976791	valid_1's auc: 0.934043
[608]	training's auc: 0.976827	valid_1's auc: 0.934048
[609]	training's auc: 0.976849	valid_1's auc: 0.934029
[610]	training's auc: 0.976882	valid_1's auc: 0.934043
[611]	training's auc: 0.976906	valid_1's auc: 0.934066
[612]	training's auc: 0.976946	valid_1's auc: 0.934076
[613]	training's auc: 0.976984	valid_1's auc: 0.934061
[614]	training's auc: 0.977017	valid_1's auc: 0.934076
[615]	training's auc: 0.977048	valid_1's auc: 0.934092
[616]	trainin

...

[36]	training's auc: 0.937168	valid_1's auc: 0.916116
[37]	training's auc: 0.937857	valid_1's auc: 0.917159
[38]	training's auc: 0.938025	valid_1's auc: 0.91691
[39]	training's auc: 0.938469	valid_1's auc: 0.917127
[40]	training's auc: 0.938499	valid_1's auc: 0.917312
[41]	training's auc: 0.938511	valid_1's auc: 0.917064
[42]	training's auc: 0.938932	valid_1's auc: 0.91766
[43]	training's auc: 0.939026	valid_1's auc: 0.917969
[44]	training's auc: 0.939236	valid_1's auc: 0.918222
[45]	training's auc: 0.939462	valid_1's auc: 0.91836
[46]	training's auc: 0.939535	valid_1's auc: 0.918358
[47]	training's auc: 0.939569	valid_1's auc: 0.91837
[48]	training's auc: 0.939868	valid_1's auc: 0.918832
[49]	training's auc: 0.940085	valid_1's auc: 0.91878
[50]	training's auc: 0.940148	valid_1's auc: 0.918663
[51]	training's auc: 0.940475	valid_1's auc: 0.919015
[52]	training's auc: 0.940572	valid_1's auc: 0.918875
[53]	training's auc: 0.940655	valid_1's auc: 0.918947
[54]	training's auc: 0.940697	val

[186]	training's auc: 0.953847	valid_1's auc: 0.922214
[187]	training's auc: 0.953938	valid_1's auc: 0.92216
[188]	training's auc: 0.954044	valid_1's auc: 0.922226
[189]	training's auc: 0.95412	valid_1's auc: 0.922223
[190]	training's auc: 0.954177	valid_1's auc: 0.922263
[191]	training's auc: 0.954257	valid_1's auc: 0.922254
[192]	training's auc: 0.954342	valid_1's auc: 0.922285
[193]	training's auc: 0.954426	valid_1's auc: 0.922403
[194]	training's auc: 0.954511	valid_1's auc: 0.922379
[195]	training's auc: 0.954611	valid_1's auc: 0.922362
[196]	training's auc: 0.95465	valid_1's auc: 0.922372
[197]	training's auc: 0.954754	valid_1's auc: 0.922416
[198]	training's auc: 0.954841	valid_1's auc: 0.922463
[199]	training's auc: 0.954922	valid_1's auc: 0.922498
[200]	training's auc: 0.954983	valid_1's auc: 0.922498
[201]	training's auc: 0.955042	valid_1's auc: 0.92248
[202]	training's auc: 0.955123	valid_1's auc: 0.92246
[203]	training's auc: 0.955216	valid_1's auc: 0.922483
[204]	training'

[335]	training's auc: 0.964512	valid_1's auc: 0.924405
[336]	training's auc: 0.964581	valid_1's auc: 0.924405
[337]	training's auc: 0.964639	valid_1's auc: 0.924404
[338]	training's auc: 0.964694	valid_1's auc: 0.924381
[339]	training's auc: 0.964734	valid_1's auc: 0.92443
[340]	training's auc: 0.964775	valid_1's auc: 0.92443
[341]	training's auc: 0.964853	valid_1's auc: 0.924453
[342]	training's auc: 0.964901	valid_1's auc: 0.924437
[343]	training's auc: 0.964968	valid_1's auc: 0.92452
[344]	training's auc: 0.96502	valid_1's auc: 0.924529
[345]	training's auc: 0.965078	valid_1's auc: 0.924526
[346]	training's auc: 0.96512	valid_1's auc: 0.924527
[347]	training's auc: 0.965167	valid_1's auc: 0.924499
[348]	training's auc: 0.965236	valid_1's auc: 0.924444
[349]	training's auc: 0.965273	valid_1's auc: 0.924473
[350]	training's auc: 0.965354	valid_1's auc: 0.92444
[351]	training's auc: 0.96544	valid_1's auc: 0.924489
[352]	training's auc: 0.9655	valid_1's auc: 0.924487
[353]	training's au

[484]	training's auc: 0.972177	valid_1's auc: 0.924916
[485]	training's auc: 0.972248	valid_1's auc: 0.924949
[486]	training's auc: 0.972297	valid_1's auc: 0.92493
[487]	training's auc: 0.97236	valid_1's auc: 0.924952
[488]	training's auc: 0.972415	valid_1's auc: 0.92493
[489]	training's auc: 0.972465	valid_1's auc: 0.924924
[490]	training's auc: 0.972496	valid_1's auc: 0.924874
[491]	training's auc: 0.972528	valid_1's auc: 0.924889
[492]	training's auc: 0.972562	valid_1's auc: 0.924899
[493]	training's auc: 0.972613	valid_1's auc: 0.924928
[494]	training's auc: 0.972652	valid_1's auc: 0.92491
[495]	training's auc: 0.972694	valid_1's auc: 0.924913
[496]	training's auc: 0.972729	valid_1's auc: 0.9249
[497]	training's auc: 0.972769	valid_1's auc: 0.92492
[498]	training's auc: 0.97281	valid_1's auc: 0.924899
[499]	training's auc: 0.97285	valid_1's auc: 0.924876
[500]	training's auc: 0.972914	valid_1's auc: 0.924876
[501]	training's auc: 0.972968	valid_1's auc: 0.924911
[502]	training's au

[633]	training's auc: 0.978241	valid_1's auc: 0.924969
[634]	training's auc: 0.978271	valid_1's auc: 0.92499
[635]	training's auc: 0.978305	valid_1's auc: 0.924972
[636]	training's auc: 0.978335	valid_1's auc: 0.924989
[637]	training's auc: 0.97836	valid_1's auc: 0.924993
[638]	training's auc: 0.978406	valid_1's auc: 0.924918
[639]	training's auc: 0.978437	valid_1's auc: 0.924956
[640]	training's auc: 0.978462	valid_1's auc: 0.924883
[641]	training's auc: 0.978504	valid_1's auc: 0.924857
[642]	training's auc: 0.978528	valid_1's auc: 0.924879
[643]	training's auc: 0.978563	valid_1's auc: 0.924847
[644]	training's auc: 0.978601	valid_1's auc: 0.924848
[645]	training's auc: 0.97864	valid_1's auc: 0.924869
[646]	training's auc: 0.978664	valid_1's auc: 0.924864
[647]	training's auc: 0.978691	valid_1's auc: 0.924892
[648]	training's auc: 0.978722	valid_1's auc: 0.924876
[649]	training's auc: 0.978774	valid_1's auc: 0.924903
[650]	training's auc: 0.978802	valid_1's auc: 0.9249
[651]	training'